### Imports

In [1]:
import pandas as pd
import datetime as dt
import gspread
from oauth2client.service_account import ServiceAccountCredentials

### Initialize Google Sheets API + Pull full sheet

In [2]:
scopes = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/gmail.compose']
creds=ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scopes)
client = gspread.authorize(creds)

In [3]:
sheet = client.open('5-3-1 Workout Weights')

In [4]:
matt = sheet.worksheet('Matt')

In [5]:
andmatt = pd.DataFrame(matt.get_all_values())
andmatt.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,,,,,,,,,,,,,,,,
1,,Today:,6/4/2017,6/4/2017,,,,,,,,,,,,
2,,Month:,1,,,,,,,,,,,,,
3,,,,,,,,,,,,,,,,
4,,,,Core Exercises,,,,,1 Rep Max Progression,,,,,,,
5,,,,165,155,225,100,,Month #,Start,Squat,Bench,Deadlift,OHP,,
6,,Week:,Set:,Squat,Bench,Deadlift,OHP,,1,5/28/17,165,155,225,100,6/24/17,1
7,,Week 1,Set 1:,110 lb x 5,105 lb x 5,150 lb x 5,65 lb x 5,,2,6/25/17,175,165,235,110,7/22/17,0
8,,,Set 2:,125 lb x 5,120 lb x 5,170 lb x 5,75 lb x 5,,3,7/23/17,185,175,245,120,8/19/17,0
9,,,Set 3:,145 lb x 5+,135 lb x 5+,195 lb x 5+,85 lb x 5+,,4,8/20/17,195,185,255,130,9/16/17,0


### Generate Main Workout DF

In [6]:
main = andmatt.loc[6:18, 1:6]
main.columns = main.iloc[0]
main = main.reset_index(drop = True).iloc[1:]
main.columns.name = None

In [7]:
main[main==""] = None
main.fillna(method='ffill', inplace = True)

In [8]:
main

,Week:,Set:,Squat,Bench,Deadlift,OHP
1,Week 1,Set 1:,110 lb x 5,105 lb x 5,150 lb x 5,65 lb x 5
2,Week 1,Set 2:,125 lb x 5,120 lb x 5,170 lb x 5,75 lb x 5
3,Week 1,Set 3:,145 lb x 5+,135 lb x 5+,195 lb x 5+,85 lb x 5+
4,Week 2,Set 1:,120 lb x 3,110 lb x 3,160 lb x 3,70 lb x 3
5,Week 2,Set 2:,135 lb x 3,125 lb x 3,180 lb x 3,80 lb x 3
6,Week 2,Set 3:,150 lb x 3+,140 lb x 3+,205 lb x 3+,90 lb x 3+
7,Week 3,Set 1:,125 lb x 5,120 lb x 5,170 lb x 5,75 lb x 5
8,Week 3,Set 2:,145 lb x 3,135 lb x 3,195 lb x 3,85 lb x 3
9,Week 3,Set 3:,160 lb x 1+,150 lb x 1+,215 lb x 1+,95 lb x 1+
10,Week 4 (Deload),Set 1:,70 lb x 5+,65 lb x 5+,90 lb x 5+,40 lb x 5+


### What Week is it again?

In [9]:
dates = andmatt.loc[5:19, 8:15]

In [10]:
dates.head(3)

,8,9,10,11,12,13,14,15
5,Month #,Start,Squat,Bench,Deadlift,OHP,,
6,1,5/28/17,165,155,225,100,6/24/17,1
7,2,6/25/17,175,165,235,110,7/22/17,0


In [11]:
dates = andmatt.loc[5:19, 8:15]
names = dates.iloc[0, 0:6].tolist() + ['End', 'Tag']
dates.columns = names
dates = dates.reset_index(drop=True).iloc[1:]

In [12]:
dates.head(3)

,Month #,Start,Squat,Bench,Deadlift,OHP,End,Tag
1,1,5/28/17,165,155,225,100,6/24/17,1
2,2,6/25/17,175,165,235,110,7/22/17,0
3,3,7/23/17,185,175,245,120,8/19/17,0


In [13]:
dcheck = dates[dates.Tag=='1'][['Start', 'End']]
dcheck = dcheck.apply(pd.to_datetime, axis=1)

In [14]:
#Python 3

def week_finder():
    for x in [1,2,3,4]:
        today = dt.datetime.today()
        start = dcheck['Start'] + dt.timedelta(weeks = x-1)
        end = dcheck['Start'] + dt.timedelta(weeks = x)
        if ((today >= start) & (today <= end)).any():
            return(str(x))
        if (today>dcheck['Start'] + dt.timedelta(weeks = 4)).any():
            raise ValueError('Date is out of Range!')

In [15]:
# #Python 2

# def week_finder():
#     for x in [1,2,3,4]:
#         today = dt.datetime.today()
#         start = dcheck['Start'][1] + dt.timedelta(weeks = x-1)
#         end = dcheck['Start'][1] + dt.timedelta(weeks = x)
#         if (today >= start) & (today <= end):
#             return(str(x))

In [16]:
week = week_finder()

In [17]:
week

'2'

### Generate This Week's Workout

In [18]:
main_f = main[main['Week:'].str.contains(week)]
main_f = main_f.drop('Week:', 1)
main_f

,Set:,Squat,Bench,Deadlift,OHP
4,Set 1:,120 lb x 3,110 lb x 3,160 lb x 3,70 lb x 3
5,Set 2:,135 lb x 3,125 lb x 3,180 lb x 3,80 lb x 3
6,Set 3:,150 lb x 3+,140 lb x 3+,205 lb x 3+,90 lb x 3+


In [19]:
main_f.index = [1,2,3]

In [20]:
main_f

,Set:,Squat,Bench,Deadlift,OHP
1,Set 1:,120 lb x 3,110 lb x 3,160 lb x 3,70 lb x 3
2,Set 2:,135 lb x 3,125 lb x 3,180 lb x 3,80 lb x 3
3,Set 3:,150 lb x 3+,140 lb x 3+,205 lb x 3+,90 lb x 3+


### Generate Accessory Workout DF

In [21]:
accessory = andmatt.loc[21:26, 3:6]
accessory.columns = main.columns[2:6]
accessory.iloc[1:].reset_index(drop = True)
accessory.columns.name = None

In [22]:
accessory

,Squat,Bench,Deadlift,OHP
21,Hack Squat,Dips,Rows,Sitting D.Bell Press
22,225lb 4x10,4x10,90lb 4x10,45lb 4x10
23,Romanian DL,Incline D.Bell Press,Lat Pull Down,Rear Delt Flys
24,90lb 4x10,45lb 4x10,80lb 4x10,25lb 4x10
25,Calf Raise,Cable Tricep Ext.,Curls,Front / Side Raises
26,75lb 4x10,80lb 4x10,30lb 4x10,17.5lb 4x10


### Subset Main Workout for Correct Week

In [23]:
week

'2'

In [24]:
week_d = dcheck.copy()
wdiff = int(week)-1
week_d['Start'] = week_d['Start'] + dt.timedelta(weeks = wdiff)
week_d['End'] = week_d['End'] + dt.timedelta(weeks = wdiff)
week_d

,Start,End
1,2017-06-04,2017-07-01


In [25]:
start = week_d['Start'][1].strftime('%m/%d/%Y')
end = week_d['End'][1].strftime('%m/%d/%Y')

### HTML Compilation

#### Generate HTML Tables with Styling

In [26]:
def recolor(val):
    '''
    adds css color elements to df
    '''
    return 'color: white ; background-color: lightskyblue'

In [27]:
a_html = (accessory.style
        .set_properties(**{'text-align': 'center',
                          'border':'1px solid',
                          'border-collapse': 'collapse',
                          'border-color': 'slategray'})
        .applymap(recolor, subset = pd.IndexSlice[[21,23,25],]))
a_html = a_html.render()

In [28]:
m_html = main_f.style.set_properties(**{'text-align': 'center',
                                       'border':'1px solid',
                                       'border-collapse': 'collapse',
                                       'border-color': 'slategray'})
m_html = m_html.render()

#### Full HTML

In [29]:
html = '''\
<html>
  <head> 
  <title> 5-3-1 Workout of the Week </title> 
  </head>
  
  <body>
  <h2>5-3-1 Workout of the Week</h2>
  
    <p>PFA - the workout of the week. It is currently <b>Week {week}</b> <br>
       <b>Week {week}</b> goes from {start} till {end}
    </p>
    
    <h4>Main Workout:</h4>
    {main_f}
    <br>
    <h4>Accessory Exercises:</h4>
    {accessory}<br>
    
  </body>
</html>
'''.format(week = week, start = start, end = end,
           main_f = m_html, accessory = a_html)

### Export to Dropbox

In [ ]:
output = open('Dropbox/1. 531 Workout.html', 'w')

In [ ]:
output.write(html)
output.close()